In [3]:
%pylab inline

2023-01-18 11:22:01,387 [585] WARNING  py.warnings:109: [JupyterRequire] /opt/conda/lib/python3.8/site-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['random']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +



Populating the interactive namespace from numpy and matplotlib


In [2]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import random
import igraph as ig
import cairocffi
#import pycairo
import hypernetx as hnx
import hypernetx.algorithms.generative_models as gm
import hypernetx.algorithms.hypergraph_modularity as hmod
import hnxwidget as hnxw

import math
import warnings
from collections import defaultdict
import pandas as pd
from hypernetx import Hypergraph

In [3]:
plt.scatter(numpy.linspace(10,10,10), numpy.linspace(10,10,10))

NameError: name 'numpy' is not defined

In [ ]:
x = randint(10)
x

In [ ]:
n = 20
m = 20 

Gr = nx.generators.erdos_renyi_graph(20, 0.2)
nx.draw(Gr)

In [ ]:
n = 25
k1 = {i : random.randint(1, 25) for i in range(n)}
k2 = {i : sorted(k1.values())[i] for i in range(n)}
H_chung = gm.chung_lu_hypergraph(k1, k2)

In [ ]:
hnx.drawing.rubber_band.draw(H_chung)

In [ ]:
n = 25
m = n
p = 0.1
H = gm.erdos_renyi_hypergraph(n, m, p)

hnx.drawing.rubber_band.draw(H)

In [4]:
G = hmod.two_section(H)
ig.plot(G)
#hnx.drawing.two_column.draw_hyper_edges(H, [0,1])

NameError: name 'H' is not defined

In [ ]:
hnx.drawing.two_column.layout_two_column(H)

In [ ]:
hnx.reports.descriptive_stats.degree_dist(H)

In [ ]:
hnx.reports.descriptive_stats.edge_size_dist(H)

In [ ]:
max(hnx.reports.descriptive_stats.edge_size_dist(H))

In [ ]:
HG = hmod.precompute_attributes(H)
HG

In [ ]:
def dcsbm_hypergraph(k1, k2, g1, g2, omega):
    """
    A function to generate an extension of DCSBM hypergraph as implemented by Mirah Shi and described for
    bipartite networks by Larremore et al. in https://doi.org/10.1103/PhysRevE.90.012805

    Parameters
    ----------
    k1 : dictionary
        This a dictionary where the keys are node ids and the values are node degrees.
    k2 : dictionary
        This a dictionary where the keys are edge ids and the values are edge degrees also known as edge sizes.
    g1 : dictionary
        This a dictionary where the keys are node ids and the values are the group ids to which the node belongs.
        The keys must match the keys of k1.
    g2 : dictionary
        This a dictionary where the keys are edge ids and the values are the group ids to which the edge belongs.
        The keys must match the keys of k2.
    omega : 2D numpy array
        This is a matrix with entries which specify the number of edges between a given node community and edge community.
        The number of rows must match the number of node communities and the number of columns
        must match the number of edge communities.


    Returns
    -------
    HyperNetX Hypergraph object

    Notes
    -----
    The sums of k1 and k2 should be the same. If they are not the same, this function returns a warning but still runs.
    The sum of k1 (and k2) and omega should be the same. If they are not the same, this function returns a warning
    but still runs and the number of entries in the incidence matrix is determined by the omega matrix.

    The output currently is a static Hypergraph object. Dynamic hypergraphs are not currently supported.

    Example::

    >>> n = 100
    >>> k1 = {i : random.randint(1, 100) for i in range(n)}
    >>> k2 = {i : sorted(k1.values())[i] for i in range(n)}
    >>> g1 = {i : random.choice([0, 1]) for i in range(n)}
    >>> g2 = {i : random.choice([0, 1]) for i in range(n)}
    >>> omega = np.array([[100, 10], [10, 100]])
    >>> H = gm.dcsbm_hypergraph(k1, k2, g1, g2, omega)
    """

    # sort dictionary by degree in decreasing order
    Nlabels = [n for n, _ in sorted(k1.items(), key=lambda d: d[1], reverse=True)]
    Mlabels = [m for m, _ in sorted(k2.items(), key=lambda d: d[1], reverse=True)]

    # these checks verify that the sum of node and edge degrees and the sum of node degrees
    # and the sum of community connection matrix differ by less than a single edge.
    if abs(sum(k1.values()) - sum(k2.values())) > 1:
        warnings.warn(
            "The sum of the degree sequence does not match the sum of the size sequence"
        )

    if abs(sum(k1.values()) - np.sum(omega)) > 1:
        warnings.warn(
            "The sum of the degree sequence does not match the entries in the omega matrix"
        )

    # get indices for each community
    community1Indices = defaultdict(list)
    for label in Nlabels:
        group = g1[label]
        community1Indices[group].append(label)

    community2Indices = defaultdict(list)
    for label in Mlabels:
        group = g2[label]
        community2Indices[group].append(label)

    bipartite_edges = list()

    kappa1 = defaultdict(lambda: 0)
    kappa2 = defaultdict(lambda: 0)
    for id, g in g1.items():
        kappa1[g] += k1[id]
    for id, g in g2.items():
        kappa2[g] += k2[id]

    for group1 in community1Indices.keys():
        for group2 in community2Indices.keys():
            # for each constant probability patch
            try:
                groupConstant = omega[group1, group2] / (
                    kappa1[group1] * kappa2[group2]
                )
            except:
                groupConstant = 0

            for u in community1Indices[group1]:
                j = 0
                v = community2Indices[group2][j]  # start from beginning every time
                # max probability
                p = min(k1[u] * k2[v] * groupConstant, 1)
                while j < len(community2Indices[group2]):
                    if p != 1:
                        r = random.random()
                        try:
                            j = j + math.floor(math.log(r) / math.log(1 - p))
                        except:
                            j = np.inf
                    if j < len(community2Indices[group2]):
                        v = community2Indices[group2][j]
                        q = min((k1[u] * k2[v]) * groupConstant, 1)
                        r = random.random()
                        if r < q / p:
                            # no duplicates
                            bipartite_edges.append((v, u))

                            p = q
                            j = j + 1

    df = pd.DataFrame(bipartite_edges)
    return Hypergraph(df, static=True)

## The output currently is a static Hypergraph object. Dynamic hypergraphs are not currently supported.

### Set is faster and almost all the graphers are working with sets.

### The order of edges does not count.

In [6]:
import string
string.ascii_letters

'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [7]:
random.choice(string.ascii_letters)

't'

In [ ]:
x = df.set_index('Number')['Data'].to_dict()
x

In [ ]:
contract_file = 

with open(contract_file, 'r', encoding='UTF-8') as f:
        f = f.read()
        dfco = json.loads(f)

# Clustering dump

In [ ]:
ABCD_groups = {}
clique_groups = {}

for i in range(1, len(assign)+1):
    ABCD_groups[i] = [int(assign[i-1])]
    clique_groups[i] = []
    
ABCD_groups

In [ ]:
edges_t,weights_t = zip(*nx.get_edge_attributes(graphs['graph_1'],'weight').items())
len(weights_t)
nx.draw(graphs['graph_1'], pos = nx.kamada_kawai_layout(G2) ,with_labels=True, edgelist=edges_t, width=weights_t)

In [ ]:
[dict(zip(val, col)) for val, col in zip(*groups_k['w_1.0'].values())]

In [ ]:
list(x for x in range(1,max(nodelist)+1))

In [ ]:
list(dict((v, [k]) for k in groups_k['w_1.0'] for v in groups_k['w_1.0'][k]).keys())
sorted(nodelist)

In [ ]:
groups_k['w_1.0']
fasz = {1: [1, 2], 2:[2, 3]}
fasz
akarmi = {}
for k in fasz:
    for v in fasz[k]:
        akarmi.setdefault(v, []).append(k)
#dict((dict.setdefault(v, []).append(k)) for k in fasz for v in fasz[k])
akarmi

In [ ]:
akarmi = dict((v, [k]) for k in groups_k['w_1.0'] for v in groups_k['w_1.0'][k])
akarmi

In [ ]:
[x for x in sorted(nodelist) if x not in (list(dict((v, [k]) for k in groups_k['w_1.0'] for v in groups_k['w_1.0'][k]).keys()))]